In [ ]:
import pickle
from keras.utils import to_categorical
import numpy as np 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import os
from keras import models
from keras import layers

def save_obj(path,obj, name ):
    with open(path+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path ,name ):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

train_images = np.zeros(1, dtype = int) 
train_labels = np.zeros(1, dtype = int)
print(train_images.shape[0])
folderpath ='C:/Data/Batch/Bactch1/Output/ModelData/'
#30% test data and 70% train data
k=0
with os.scandir(folderpath) as entries:
    for entry in entries:
      
        name = entry.name.split('.')[0]
        data =  load_obj(folderpath,name)
        data_img = np.array(data)
        #input_data = np.expand_dims(data_img, axis=3)
        length =len(data)
        input_data = data_img.reshape(length, 27*27)
        #print(input_data.shape)
        countTrain = round(length * 0.1)
        counttest = length - countTrain
        #print(countTrain)
        if train_images.shape[0] == 1:
            train_images = input_data[0:countTrain,:]
            train_labels = np.array([k for i in range(countTrain)])
            test_images =  input_data[countTrain:,:]
            test_labels = np.array([k for i in range(counttest)])
        else:
            train_images = np.concatenate((train_images, input_data[0:countTrain,:]))
            train_labels = np.concatenate((train_labels, np.array([k for i in range(countTrain)])))
            test_images = np.concatenate((test_images, input_data[countTrain:,:]))
            test_labels = np.concatenate((test_labels, np.array([k for i in range(counttest)])))
        k = k + 1    
        print(name)

path = 'C:/Data/Batch/Bactch1/Output/Clubbeddata/'
save_obj(path,train_images,'train_images')

save_obj(path,train_labels,'train_labels')

save_obj(path,test_images,'test_images')

save_obj(path,test_labels,'test_labels')


In [13]:
path = 'C:/Data/Batch/Bactch1/Output/Clubbeddata/'

train_images = load_obj(path,'train_images')
train_labels =  load_obj(path,'train_labels')
test_images = load_obj(path,'test_images')
test_labels = load_obj(path,'test_labels')

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

# test_images = train_images[:10000]
# test_labels  = train_labels[:10000]

# predict_images =  train_images[10000:]
# predict_labels =  train_labels[10000:]

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model = models.Sequential()
#model.add(layers.Flatten())
model.add(layers.Dense(150, activation='relu',input_shape=(27*27,)))
model.add(layers.Dense(150, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))
model.summary()

epochs = 10
batchsize = 100
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batchsize,
                   validation_data=(test_images, test_labels))

results = model.evaluate(test_images, test_labels)

print(epochs)
print(batchsize)
print(model.metrics_names)
print(results)
print(history.history.keys())

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


MemoryError: 